In [5]:
!pip install kiteconnect

In [6]:
from kiteconnect import KiteConnect

# logging.basicConfig(level=logging.DEBUG)
print('here')
kite = KiteConnect(api_key="aygbpvrrcsaybucb")

# Step 1 - Open the below URL and login and collect the request token
# https://kite.zerodha.com/connect/login?v=4&api_key=aygbpvrrcsaybucb
# Step 2 - Run this program and provide the request token
# Step 3 - You would get access token
# Step 4 - Update the access token in KiteSettings.py

# request_token = input("Request Token: ")
# data = kite.generate_session(request_token, api_secret="ant7m5o74v4x0sptg18gx18kzbkl6i5x")
kite.set_access_token("0ktv2vzfBlzqukwQ9M6EqcJx4zZEs5xe")

# print("====================")
# print("Access Token: ",data["access_token"])

here


In [7]:
import pandas as pd
nse = kite.instruments('NSE')
nse_data = pd.DataFrame(nse)
nse_data

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.00,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.00,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.00,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.00,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.00,0,EQ,INDICES,NSE
...,...,...,...,...,...,...,...,...,...,...,...,...
8035,194190337,758556,LTFL28-N0,,0.0,,0.0,0.01,1,EQ,NSE,NSE
8036,194190849,758558,KALPATARU,KALPATARU,0.0,,0.0,0.05,1,EQ,NSE,NSE
8037,194192129,758563,ELLEN,ELLENBARRIE INDUS GASES L,0.0,,0.0,0.05,1,EQ,NSE,NSE
8038,194194177,758571,SHKSIL-ST,SHRI HARE-KRI SPO IRON L,0.0,,0.0,0.05,2000,EQ,NSE,NSE


In [8]:
def calculate_transaction_charge_buy(quantity, price):
    total_transaction_charge = 0
    total_amount = quantity * price
    brokerage = min(20, 0.0003 * total_amount)
    transaction_charge = 0.0000325 * total_amount
    sebi_charges = 0.000001 * total_amount
    stamp_duty = 0.00003*total_amount
    gst = 0.18 * (brokerage + sebi_charges + transaction_charge)
    total_transaction_charge += brokerage + stamp_duty + transaction_charge + gst + sebi_charges
    return total_transaction_charge

def calculate_transaction_charge_sell(quantity, price):
    total_transaction_charge = 0
    total_amount = quantity * price
    brokerage = min(20, 0.0003 * total_amount)
    stt = 0.00025 * (total_amount)
    transaction_charge = 0.0000325 * total_amount
    sebi_charges = 0.000001 * total_amount
    gst = 0.18 * (brokerage + sebi_charges + transaction_charge)
    total_transaction_charge += brokerage + stt + transaction_charge + gst + sebi_charges
    return total_transaction_charge

In [9]:
import pandas as pd
import yfinance as yf

In [10]:
import warnings

In [11]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [12]:
def final_backtest(IA_buy, IA_sell, profit_margin,stop_loss, stock_data_file):

    result_df = pd.DataFrame(columns=['Date','Buy_stocks','Sell_stocks' , 'Status_L1', 'TP_L1', 'SL_L1', 'Status_L2', 'TP_L2', 'SL_L2', 'Status_L3', 'TP_L3', 'SL_L3', 'Status_S1', 'TP_S1', 'SL_S1', 'Status_S2', 'TP_S2', 'SL_S2', 'Status_S3', 'TP_S3', 'SL_S3', 'IA_buy','IA_sell','buy_profit','sell_profit','Transaction_Charges','Profit_after_Transaction'])

    # Read stock data from CSV file
    stock_data = pd.read_csv(stock_data_file)
    profit_margin=profit_margin/5
    stop_loss=stop_loss/5
    # # Add ".NS" to stock names
    stock_data['Buy'] = stock_data['Buy'].apply(lambda x: [stock + '' for stock in eval(x)])
    stock_data['Sell'] = stock_data['Sell'].apply(lambda x: [stock + '' for stock in eval(x)])

    # Iterate over each date
    for i in range(len(stock_data)):
        n=3
        date = pd.to_datetime(stock_data.loc[i, 'Date'], dayfirst=True)
        buy_amount=IA_buy
        sell_amount=IA_sell
        buy_amount_per_stock=buy_amount/3
        sell_amount_per_stock=sell_amount/3
        buy_list = stock_data.loc[i, 'Buy']
        sell_list = stock_data.loc[i, 'Sell']

        tp = profit_margin
        ls = stop_loss
        status_long=[]
        status_short=[]
        TP_long =[]
        TP_short=[]
        SL_long =[]
        SL_short=[]
        buy_profit=0
        sell_profit=0
        total_transaction=0
        # print(buy_list)
        for stock in buy_list:
            # print(stock)
            token = nse_data[nse_data.tradingsymbol == stock].instrument_token.values[0]
            #print(date)
            df = kite.historical_data(token, str(date)[:10] + " 09:15:00", str(date)[:10] + " 15:14:00", "minute", False, False)
            df = pd.DataFrame(df)
            if(df.size == 0):
                continue
            traded_close = df['open'][0]
            tp_key=0
            sl_key=0
            status_key=-2 #Initial Stage
            total_transaction+=calculate_transaction_charge_buy(((buy_amount_per_stock*5)/traded_close),traded_close)
            for j in range(len(df)):
                if(j<60):
                  ls=0.02
                else:
                  ls=0.003
                if((df.loc[j, 'high']-traded_close)/traded_close >= tp and tp_key==0):
                    if(status_key==-2):
                        ltp=df.loc[j,'high']
                        status_long.append(1)
                        status_key=1
                        total_transaction+=calculate_transaction_charge_sell(((buy_amount_per_stock*5)/traded_close),ltp)
                        buy_profit+=tp*buy_amount_per_stock*5
                        # buy_profit+=(ltp-traded_close)*((buy_amount_per_stock*5)/traded_close)
                    TP_long.append(df.loc[j, 'date'])
                    tp_key=1
                if((traded_close-df.loc[j, 'low'])/traded_close >=ls and sl_key==0):
                    if(status_key==-2):
                        ltp=df.loc[j,'low']
                        status_long.append(0)
                        status_key=0
                        total_transaction+=calculate_transaction_charge_sell(((buy_amount_per_stock*5)/traded_close),ltp)
                        # buy_profit-=ls*buy_amount_per_stock*5
                        buy_profit-=(traded_close-ltp)*((buy_amount_per_stock*5)/traded_close)
                    SL_long.append(df.loc[j, 'date'])
                    sl_key=1
            if(tp_key==0 and sl_key==0):
                status_long.append(-1)
                total_transaction+=calculate_transaction_charge_sell(((buy_amount_per_stock*5)/traded_close),traded_close)
                ltp = df.loc[-1,'close']
                buy_profit+=(ltp-traded_close)*((buy_amount_per_stock*5)/traded_close)
                TP_long.append(-1)
                SL_long.append(-1)
            elif(tp_key==0):
                TP_long.append(-1)
            elif(sl_key==0):
                SL_long.append(-1)

        for stock in sell_list:
            token = nse_data[nse_data.tradingsymbol == stock].instrument_token.values[0]
            df = kite.historical_data(token, str(date)[:10] + " 09:15:00", str(date)[:10] + " 15:14:00", "minute", False, False)
            df = pd.DataFrame(df)
            if(df.size == 0):
                continue
            traded_close = df['open'][0]
            tp_key=0
            sl_key=0
            status_key=-2 #Initial Stage
            total_transaction+=calculate_transaction_charge_sell(((sell_amount_per_stock*5)/traded_close),traded_close)
            for j in range(len(df)):
                if(j<60):
                  ls=0.02
                else:
                  ls=0.003
                if((traded_close-df.loc[j, 'low'])/traded_close >= tp and tp_key==0):
                    if(status_key==-2):
                        ltp=df.loc[j,'low']
                        status_short.append(1)
                        status_key=1
                        total_transaction+=calculate_transaction_charge_buy(((sell_amount_per_stock*5)/traded_close),ltp)
                        # sell_profit+=(traded_close-ltp)*((sell_amount_per_stock*5)/traded_close)
                        sell_profit+=tp*sell_amount_per_stock*5
                    TP_short.append(df.loc[j, 'date'])
                    tp_key=1
                if((df.loc[j, 'high']-traded_close)/traded_close >=ls and sl_key==0):
                    if(status_key==-2):
                        ltp=df.loc[j,'high']
                        status_short.append(0)
                        status_key=0
                        total_transaction+=calculate_transaction_charge_buy(((sell_amount_per_stock*5)/traded_close),ltp)
                        # sell_profit-=ls*sell_amount_per_stock*5
                        sell_profit-=(ltp-traded_close)*((sell_amount_per_stock*5)/traded_close)
                    SL_short.append(df.loc[j, 'date'])
                    sl_key=1
            if(tp_key==0 and sl_key==0):
                status_short.append(-1)
                total_transaction+=calculate_transaction_charge_buy(((sell_amount_per_stock*5)/traded_close),traded_close)
                ltp = df.loc[-1,'close']
                sell_profit+=(traded_close-ltp)*((sell_amount_per_stock*5)/traded_close)
                TP_short.append(-1)
                SL_short.append(-1)
            elif(tp_key==0):
                TP_short.append(-1)
            elif(sl_key==0):
                SL_short.append(-1)
        #print("YESS")
        status_long_df = pd.DataFrame(status_long)
        if(status_long_df.size == 0):
            continue
        total_profit=buy_profit+sell_profit
        Pro_after_Transaction=total_profit-total_transaction
        IA_buy+=Pro_after_Transaction/2
        IA_sell+=Pro_after_Transaction/2
        result_df = result_df.append({'Date': date,'Buy_stocks': buy_list, 'Sell_stocks': sell_list,
                                      'Status_L1': status_long[0], 'TP_L1':TP_long[0], 'SL_L1':SL_long[0],
                                      'Status_L2': status_long[1], 'TP_L2':TP_long[1], 'SL_L2':SL_long[1],
                                      'Status_L3': status_long[2], 'TP_L3':TP_long[2], 'SL_L3':SL_long[2],
                                      'Status_S1': status_short[0], 'TP_S1':TP_short[0], 'SL_S1':SL_short[0],
                                      'Status_S2': status_short[1], 'TP_S2':TP_short[1], 'SL_S2':SL_short[1],
                                      'Status_S3': status_short[2], 'TP_S3':TP_short[2], 'SL_S3':SL_short[2],
                                      'IA_buy':buy_amount, 'IA_sell': sell_amount, 'buy_profit': buy_profit, 'sell_profit': sell_profit, 'Transaction_Charges':total_transaction, 'Profit_after_Transaction': Pro_after_Transaction},  ignore_index=True)
        print(date)

    return result_df



In [ ]:
stock_data_file = '/content/nifty200_3.csv'

result = final_backtest(60000,60000,0.01313,0.015,stock_data_file)

result

In [ ]:
result.to_csv('/content/nifty200_v4.csv', index=False)